In [7]:
# Importing necessary libraries
import pandas as pd
pd.set_option('display.max_rows', None)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
#from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
df_full_numeric = ['age', 'bmi', 'height', 'pre_icu_los_days', 'heart_rate_apache', 'map_apache',
                   'resprate_apache', 'temp_apache', 'd1_potassium_min', 'd1_potassium_max', 'h1_mbp_max', 
                   'h1_mbp_min', 'h1_sysbp_max', 'd1_glucose_min',
                   'd1_glucose_max', 'h1_resprate_max', 'h1_resprate_min', 'h1_heartrate_max', 
                   'd1_temp_max', 'd1_temp_min',  'd1_resprate_min', 
                   'd1_resprate_max', 'd1_spo2_min', 'd1_spo2_max', 'h1_spo2_max', 'h1_spo2_min', 'd1_mbp_min', 
                   'd1_mbp_max', 'd1_sysbp_min', 'd1_sysbp_max', 'd1_heartrate_min', 
                   'apache_4a_hospital_death_prob']

### Survival Model

In [9]:
survival_model = pd.read_csv('survival_clean.csv')
survival_model.head()

age        bmi  elective_surgery  ethnicity gender  height  \
0  68.0  22.730000                 0  Caucasian      M   180.3   
1  77.0  27.420000                 0  Caucasian      F   160.0   
2  25.0  31.950000                 0  Caucasian      F   172.7   
3  81.0  22.640000                 1  Caucasian      F   165.1   
4  19.0  27.654655                 0  Caucasian      M   188.0   

            icu_admit_source icu_stay_type      icu_type  pre_icu_los_days  \
0                      Floor         admit         CTICU              0.54   
1                      Floor         admit  Med-Surg ICU              0.93   
2       Accident & Emergency         admit  Med-Surg ICU              0.00   
3  Operating Room / Recovery         admit         CTICU              0.00   
4       Accident & Emergency         admit  Med-Surg ICU              0.07   

   ...  cirrhosis  diabetes_mellitus  hepatic_failure  immunosuppression  \
0  ...        0.0                1.0              0.0                0.0   
1  ...        0.0                1.0              0.0                0.0   
2  ...        0.0                0.0              0.0                0.0   
3  ...        0.0                0.0              0.0                0.0   
4  ...        0.0                0.0              0.0                0.0   

   leukemia  lymphoma  solid_tumor_with_metastasis  apache_3j_bodysystem  \
0       0.0       0.0                          0.0                Sepsis   
1       0.0       0.0                          0.0           Respiratory   
2       0.0       0.0                          0.0             Metabolic   
3       0.0       0.0                          0.0        Cardiovascular   
4       0.0       0.0                          0.0                Trauma   

   apache_2_bodysystem  hospital_death  
0       Cardiovascular               0  
1          Respiratory               0  
2            Metabolic               0  
3       Cardiovascular               0  
4               Trauma               0  

[5 rows x 66 columns]

In [10]:
import pandas as pd

# Assuming `df` is your DataFrame and `df_full_numeric` is your list of numeric columns
# Calculate the correlation matrix for numeric columns
correlation_matrix = survival_model[df_full_numeric].corr()

# Identify pairs with correlation > 0.8 (absolute value)
high_correlation_pairs = correlation_matrix.abs().unstack().sort_values(ascending=False)
high_correlation_pairs = high_correlation_pairs[high_correlation_pairs != 1]  # Exclude self-correlation

# Select only pairs with correlation above 0.8 and remove duplicates
high_corr_pairs_filtered = high_correlation_pairs[high_correlation_pairs > 0.8].drop_duplicates()

# Display the high correlation pairs
print(high_corr_pairs_filtered.head(20))


Series([], dtype: float64)


In [11]:
# List of columns to drop based on high correlation
columns_to_drop = [
    'apache_4a_icu_death_prob',   # Keep apache_4a_hospital_death_prob
    'weight',                     # Keep bmi
    'h1_diasbp_min',              # Keep h1_mbp_min
    'h1_heartrate_min',           # Keep h1_heartrate_max
    'd1_diasbp_min',              # Keep d1_mbp_min
    'h1_diasbp_max',              # Keep h1_mbp_max
    'd1_diasbp_max',              # Keep d1_mbp_max
    'h1_sysbp_min',               # Keep h1_mbp_min
    'd1_heartrate_max'            # Keep heart_rate_apache
]

# Drop these columns from the DataFrame
survival_model_v1 = survival_model.drop(columns=columns_to_drop)

# Verify the columns have been removed
print("Remaining columns after removal:")
print(survival_model_v1.columns)


Remaining columns after removal:
Index(['age', 'bmi', 'elective_surgery', 'ethnicity', 'gender', 'height',
       'icu_admit_source', 'icu_stay_type', 'icu_type', 'pre_icu_los_days',
       'apache_post_operative', 'arf_apache', 'gcs_eyes_apache',
       'gcs_motor_apache', 'gcs_unable_apache', 'gcs_verbal_apache',
       'heart_rate_apache', 'intubated_apache', 'map_apache',
       'resprate_apache', 'temp_apache', 'ventilated_apache',
       'd1_heartrate_min', 'd1_mbp_max', 'd1_mbp_min', 'd1_resprate_max',
       'd1_resprate_min', 'd1_spo2_max', 'd1_spo2_min', 'd1_sysbp_max',
       'd1_sysbp_min', 'd1_temp_max', 'd1_temp_min', 'h1_heartrate_max',
       'h1_mbp_max', 'h1_mbp_min', 'h1_resprate_max', 'h1_resprate_min',
       'h1_spo2_max', 'h1_spo2_min', 'h1_sysbp_max', 'd1_glucose_max',
       'd1_glucose_min', 'd1_potassium_max', 'd1_potassium_min',
       'apache_4a_hospital_death_prob', 'aids', 'cirrhosis',
       'diabetes_mellitus', 'hepatic_failure', 'immunosuppression', 'l

In [12]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# Define columns based on your dataset
df_full_categorical = ['apache_3j_bodysystem', 'apache_2_bodysystem', 'ethnicity', 
                       'icu_admit_source', 'gender', 'icu_stay_type', 'icu_type']
df_full_binary = ['elective_surgery', 'arf_apache', 'gcs_unable_apache', 'intubated_apache', 'ventilated_apache',
                  'aids', 'cirrhosis', 'diabetes_mellitus', 'hepatic_failure', 'immunosuppression', 'leukemia',
                  'lymphoma', 'solid_tumor_with_metastasis', 'apache_post_operative']
df_full_multicat = ['gcs_eyes_apache', 'gcs_motor_apache', 'gcs_verbal_apache']
df_full_numeric = ['age', 'bmi', 'height', 'pre_icu_los_days', 'heart_rate_apache', 'map_apache',
                   'resprate_apache', 'temp_apache', 'd1_potassium_min', 'd1_potassium_max', 'h1_mbp_max', 
                   'h1_mbp_min', 'h1_sysbp_max', 'd1_glucose_min',
                   'd1_glucose_max', 'h1_resprate_max', 'h1_resprate_min', 'h1_heartrate_max', 
                   'd1_temp_max', 'd1_temp_min',  'd1_resprate_min', 
                   'd1_resprate_max', 'd1_spo2_min', 'd1_spo2_max', 'h1_spo2_max', 'h1_spo2_min', 'd1_mbp_min', 
                   'd1_mbp_max', 'd1_sysbp_min', 'd1_sysbp_max', 'd1_heartrate_min', 
                   'apache_4a_hospital_death_prob']

# Exclude the target column
X = survival_model_v1.drop(columns=['hospital_death'])

# Define the preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), df_full_categorical),
        ('multi', OneHotEncoder(drop='first', handle_unknown='ignore'), df_full_multicat),
        ('num', 'passthrough', df_full_numeric),
        ('binary', 'passthrough', df_full_binary)
    ]
)

# Apply preprocessing transformations
X_processed = preprocessor.fit_transform(X)

# Convert the processed array back to a DataFrame for VIF calculation
transformed_columns = preprocessor.get_feature_names_out()
X_processed_df = pd.DataFrame(X_processed, columns=transformed_columns)

# Identify numeric columns in the transformed data
numeric_transformed_columns = [col for col in transformed_columns if any(num_col in col for num_col in df_full_numeric)]

# Standardize the identified numeric columns
scaler = StandardScaler()
X_processed_df[numeric_transformed_columns] = scaler.fit_transform(X_processed_df[numeric_transformed_columns])

# Check the processed DataFrame
X_processed_df.head()

cat__apache_3j_bodysystem_Gastrointestinal  \
0                                         0.0   
1                                         0.0   
2                                         0.0   
3                                         0.0   
4                                         0.0   

   cat__apache_3j_bodysystem_Genitourinary  \
0                                      0.0   
1                                      0.0   
2                                      0.0   
3                                      0.0   
4                                      0.0   

   cat__apache_3j_bodysystem_Gynecological  \
0                                      0.0   
1                                      0.0   
2                                      0.0   
3                                      0.0   
4                                      0.0   

   cat__apache_3j_bodysystem_Hematological  \
0                                      0.0   
1                                      0.0   
2                                      0.0   
3                                      0.0   
4                                      0.0   

   cat__apache_3j_bodysystem_Metabolic  \
0                                  0.0   
1                                  0.0   
2                                  1.0   
3                                  0.0   
4                                  0.0   

   cat__apache_3j_bodysystem_Musculoskeletal/Skin  \
0                                             0.0   
1                                             0.0   
2                                             0.0   
3                                             0.0   
4                                             0.0   

   cat__apache_3j_bodysystem_Neurological  \
0                                     0.0   
1                                     0.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   

   cat__apache_3j_bodysystem_Respiratory  cat__apache_3j_bodysystem_Sepsis  \
0                                    0.0                               1.0   
1                                    1.0                               0.0   
2                                    0.0                               0.0   
3                                    0.0                               0.0   
4                                    0.0                               0.0   

   cat__apache_3j_bodysystem_Trauma  ...  binary__ventilated_apache  \
0                               0.0  ...                        0.0   
1                               0.0  ...                        1.0   
2                               0.0  ...                        0.0   
3                               0.0  ...                        1.0   
4                               1.0  ...                        0.0   

   binary__aids  binary__cirrhosis  binary__diabetes_mellitus  \
0           0.0                0.0                        1.0   
1           0.0                0.0                        1.0   
2           0.0                0.0                        0.0   
3           0.0                0.0                        0.0   
4           0.0                0.0                        0.0   

   binary__hepatic_failure  binary__immunosuppression  binary__leukemia  \
0                      0.0                        0.0               0.0   
1                      0.0                        0.0               0.0   
2                      0.0                        0.0               0.0   
3                      0.0                        0.0               0.0   
4                      0.0                        0.0               0.0   

   binary__lymphoma  binary__solid_tumor_with_metastasis  \
0               0.0                                  0.0   
1               0.0                                  0.0   
2               0.0                                  0.0   
3               0.0                                  0.0

In [13]:
from sklearn.feature_selection import VarianceThreshold

# Remove constant columns (near-zero variance)
vt = VarianceThreshold(threshold=1e-5)
X_reduced = vt.fit_transform(X_processed_df)
X_reduced_df = pd.DataFrame(X_reduced, columns=X_processed_df.columns[vt.get_support()])

# Remove duplicate columns (to avoid perfect multicollinearity)
X_reduced_df = X_reduced_df.loc[:, ~X_reduced_df.T.duplicated()]

# Standardize numeric columns
numeric_transformed_columns = [col for col in transformed_columns if any(num_col in col for num_col in df_full_numeric)]
scaler = StandardScaler()
X_reduced_df[numeric_transformed_columns] = scaler.fit_transform(X_reduced_df[numeric_transformed_columns])

# Function to calculate VIF and iteratively remove features with high VIF
def calculate_vif(df, threshold=5.0):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif_data

# Iteratively remove features with VIF > threshold
while True:
    vif_df = calculate_vif(X_reduced_df)
    max_vif = vif_df['VIF'].max()
    
    if max_vif > 5.0:  # Set threshold for VIF
        # Identify the feature with the highest VIF
        feature_to_drop = vif_df.sort_values(by='VIF', ascending=False).iloc[0]['Feature']
        print(f"Dropping '{feature_to_drop}' with VIF: {max_vif}")
        
        # Drop the feature with the highest VIF from the dataframe
        X_reduced_df = X_reduced_df.drop(columns=[feature_to_drop])
    else:
        break

# Display final features and their VIFs
print("Final set of features and VIFs:")
print(calculate_vif(X_reduced_df))

Dropping 'cat__apache_2_bodysystem_Renal/Genitourinary' with VIF: 99.84121827819966
Dropping 'multi__gcs_motor_apache_6.0' with VIF: 45.749088699032136
Dropping 'multi__gcs_eyes_apache_4.0' with VIF: 18.84534991960476
Dropping 'binary__apache_post_operative' with VIF: 11.837932329718853
Dropping 'cat__ethnicity_Caucasian' with VIF: 7.297610700442627
Dropping 'cat__icu_type_Med-Surg ICU' with VIF: 6.070907564538821
Dropping 'cat__icu_admit_source_Operating Room / Recovery' with VIF: 5.694445259778659
Final set of features and VIFs:
                                           Feature       VIF
0       cat__apache_3j_bodysystem_Gastrointestinal  1.397072
1          cat__apache_3j_bodysystem_Genitourinary  1.091610
2          cat__apache_3j_bodysystem_Gynecological  1.037064
3          cat__apache_3j_bodysystem_Hematological  1.032835
4              cat__apache_3j_bodysystem_Metabolic  1.503967
5   cat__apache_3j_bodysystem_Musculoskeletal/Skin  1.909029
6           cat__apache_3j_bodysyste

In [14]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.model_selection import train_test_split

# Assuming `X_reduced_df` is the DataFrame after handling VIF and multicollinearity

# Step 1: Map ICU stay durations to `apache_3j_bodysystem` categories
icu_stay_mapping = {
    'Cardiovascular': 3.5,
    'Gastrointestinal': 5,
    'Genitourinary': 10,
    'Gynecological': 4,
    'Hematological': 6,
    'Metabolic': 31,
    'Musculoskeletal/Skin': 3,
    'Neurological': 3.5,
    'Respiratory': 4,
    'Sepsis': 10,
    'Trauma': 15,
    'Haematologic': 4.4,
    'Neurologic': 3.5,
    'Renal': 4.2
}

# Assume `survival_model_v1` contains 'apache_3j_bodysystem' and 'pre_icu_los_days' columns for the transformation.
# Merge `X_reduced_df` with the `apache_3j_bodysystem` and `pre_icu_los_days` columns for continuity
df = X_reduced_df.copy()
df['apache_3j_bodysystem'] = survival_model_v1['apache_3j_bodysystem']
df['pre_icu_los_days'] = survival_model_v1['pre_icu_los_days']

# Step 2: Apply ICU stay mapping to calculate `total_icu_los_days`
df['average_icu_stay'] = df['apache_3j_bodysystem'].map(icu_stay_mapping)

# Calculate `total_icu_los_days` by adding `pre_icu_los_days` to `average_icu_stay`
df['total_icu_los_days'] = df['pre_icu_los_days'] + df['average_icu_stay']

# Set this column as `time_to_event`
df['time_to_event'] = df['total_icu_los_days']

# Drop intermediate columns that are no longer needed
df = df.drop(columns=['pre_icu_los_days', 'average_icu_stay', 'apache_3j_bodysystem'])

# Step 3: Set up the event column and finalize survival data
df['event'] = survival_model_v1['hospital_death']  # Assuming 'hospital_death' is the event column in original data

# Step 4: Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['event'])

# Step 5: Define predictors, excluding `time_to_event` and `event`
predictors = train_df.columns.difference(['time_to_event', 'event'])

# Step 6: Fit the Cox Proportional Hazards model
cox_model = CoxPHFitter(penalizer=0.1)  # Adding regularization for better convergence

# Fit the model on the training data
cox_model.fit(train_df[['time_to_event', 'event'] + predictors.tolist()], 
              duration_col='time_to_event', event_col='event')
cox_model.print_summary()

# Step 7: Evaluate model performance on test data
cox_hazards = cox_model.predict_partial_hazard(test_df)
concordance_cox = concordance_index(test_df['time_to_event'], -cox_hazards, test_df['event'])
print(f"Cox Proportional Hazards Concordance Index on Test Set: {concordance_cox}")


<lifelines.CoxPHFitter: fitted with 73370 total observations, 67038 right-censored observations>
             duration col = 'time_to_event'
                event col = 'event'
                penalizer = 0.1
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 73370
number of events observed = 6332
   partial log-likelihood = -59054.63
         time fit was run = 2024-12-06 22:34:38 UTC

---
                                                coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                                                       
binary__aids                                    0.06      1.06      0.26           -0.45            0.56                0.64                1.76
binary__arf_apache                              0.13      1.14      0.05            0.04            0.23                1.04                1.25
binary__cirrhosis                               0.12      1.12      0.06           -0.01            0.24                0.99                1.27
binary__diabetes_mellitus                      -0.09      0.92      0.02           -0.13           -0.05                0.88                0.95
binary__elective_surgery                       -0.26      0.77      0.03           -0.31           -0.21                0.74                0.81
binary__gcs_unable_apache                       0.41      1.50      0.07            0.27            0.55                1.31                1.73
binary__hepatic_failure                         0.18      1.20      0.07            0.05            0.31                1.05                1.37
binary__immunosuppression                       0.14      1.15      0.05            0.04            0.23                1.04                1.26
binary__intubated_apache                        0.18      1.20      0.02            0.14            0.23                1.15                1.25
binary__leukemia                                0.04      1.04      0.09           -0.13            0.21                0.88                1.23
binary__lymphoma                                0.13      1.14      0.11           -0.09            0.36                0.91                1.43
binary__solid_tumor_with_metastasis             0.23      1.26      0.05            0.12            0.33                1.13                1.39
binary__ventilated_apache                       0.35      1.42      0.02            0.31            0.39                1.37                1.48
cat__apache_2_bodysystem_Undefined Diagnoses    0.38      1.46      0.17            0.04            0.72                1.04                2.05
cat__apache_2_bodysystem_Undefined diagnoses   -0.22      0.80      0.05           -0.32           -0.12                0.73                0.88
cat__apache_3j_bodysystem_Gastrointestinal     -0.11      0.89      0.03           -0.17           -0.05                0.84                0.95
cat__apache_3j_bodysystem_Genitourinary        -0.51      0.60      0.05           -0.62           -0.41                0.54                0.66
cat__apache_3j_bodysystem_Gynecological        -0.23      0.79      0.18           -0.58            0.11                0.56                1.12
cat__apache_3j_bodysystem_Hematological        -0.09      0.91      0.10           -0.29            0.10                0.75                1.11
cat__apache_3j_bodysystem_Metabolic            -0.93      0.39      0.04           -1.00           -0.86                0.37                0.42
cat__apache_3j_bodysystem_Musculoskeletal/Skin  0.25      1.29      0.09            0.07            0.44                1.07                1.55
cat__apache_3j_bodysystem_Neurological          0.41      1.51      0.03            0.36            0.47                1.43                1.60
cat__apache_3j_bodysystem_Respiratory           0.13      1.13      0.03         

Cox Proportional Hazards Concordance Index on Test Set: 0.9133114213869873


In [15]:
hazard_ratios = cox_model.summary[['exp(coef)']]
print("Hazard Ratios (exp(coef)):\n")
print(hazard_ratios)

Hazard Ratios (exp(coef)):

                                                exp(coef)
covariate                                                
binary__aids                                     1.059030
binary__arf_apache                               1.142394
binary__cirrhosis                                1.122911
binary__diabetes_mellitus                        0.915152
binary__elective_surgery                         0.773083
binary__gcs_unable_apache                        1.501344
binary__hepatic_failure                          1.196651
binary__immunosuppression                        1.148402
binary__intubated_apache                         1.197649
binary__leukemia                                 1.037546
binary__lymphoma                                 1.141532
binary__solid_tumor_with_metastasis              1.256446
binary__ventilated_apache                        1.420453
cat__apache_2_bodysystem_Undefined Diagnoses     1.462672
cat__apache_2_bodysystem_Undefined diagnoses

In [16]:
hazard_ratios_p = cox_model.summary[['p']]
print("Hazard Ratios p values:\n")
print(hazard_ratios_p)

Hazard Ratios p values:

                                                            p
covariate                                                    
binary__aids                                     8.243782e-01
binary__arf_apache                               5.123710e-03
binary__cirrhosis                                6.502243e-02
binary__diabetes_mellitus                        2.674688e-05
binary__elective_surgery                         1.431844e-24
binary__gcs_unable_apache                        1.078897e-08
binary__hepatic_failure                          8.434899e-03
binary__immunosuppression                        4.280718e-03
binary__intubated_apache                         3.233337e-15
binary__leukemia                                 6.686518e-01
binary__lymphoma                                 2.489318e-01
binary__solid_tumor_with_metastasis              1.502806e-05
binary__ventilated_apache                        1.465392e-70
cat__apache_2_bodysystem_Undefined Diagnoses 